# Convert LAMMPS DUMP file to VASP POSCAR file

## Header

In [ ]:
import os
import numpy as np

print('This script converts a single-frame LAMMPS DUMP file into a VASP POSCAR file.')
print('This script is interactive and requires user input. Please read carefully before proceeding:\n')

print('The lattice may need to be reoptimized using DFT. As such, all atoms are printed in direct coordinates.')
print('For reverse conversion to LAMMPS DATA file, use poscar2lmp.awk\n')

## Extract unit cell information

In [ ]:
pwd = os.getcwd()

name = input('Enter the name of the LAMMPS DUMP file (id, type, x, y, z) to be converted : ')
file = pwd + '/' + name
log = open(file,'r')
log_lines = log.readlines()
log_lines = [line.split() for line in log_lines]

for line_index, line in enumerate(log_lines):
    for l in line:
        
        if l == 'NUMBER':
            N_atom = int(log_lines[line_index+1][0])    # Number of atoms
        
        elif l == 'BOX':
            
            xlo_bound = float(log_lines[line_index+1][0])    # Box bounds
            xhi_bound = float(log_lines[line_index+1][1])
            xy = float(log_lines[line_index+1][2])
            
            ylo_bound = float(log_lines[line_index+2][0])
            yhi_bound = float(log_lines[line_index+2][1])
            xz = float(log_lines[line_index+2][2])
            
            zlo_bound = float(log_lines[line_index+3][0])
            zhi_bound = float(log_lines[line_index+3][1])
            yz = float(log_lines[line_index+3][2])
            
            xlo = xlo_bound - np.min(np.array([0.0, xy, xz, xy+xz]))
            xhi = xhi_bound - np.max(np.array([0.0, xy, xz, xy+xz]))
            ylo = ylo_bound - np.min(np.array([0.0, yz]))
            yhi = yhi_bound - np.max(np.array([0.0, yz]))
            zlo = zlo_bound
            zhi = zhi_bound
        
        elif l == 'id':
            start = line_index + 1
        
for line in log_lines[start:]:
    line[:2] = np.array(line[:2]).astype(int)    # Atom ID, atom type
    line[2:5] = np.array(line[2:5]).astype(float)    # x, y, z

log.close()

lat = np.array([[xhi-xlo, 0.0, 0.0], [xy, yhi-ylo, 0.0], [xz, yz, zhi-zlo]])    # Lattice matrix

print('Unit cell information extracted.\n')

## Extract system information

In [ ]:
sys = input('Enter the system name : ')

a = ' '.join(str(l) for l in lat[0])
b = ' '.join(str(l) for l in lat[1])
c = ' '.join(str(l) for l in lat[2])

scale = 1.00000000000000    # Assumes no lattice scaling

ls_element = input('Enter the list of the elements, separated by space, in the same order as LAMMPS atom types : ')

ls_type = []    # List of atom types
for line in log_lines[start:]:
    Type = line[1]
    if Type not in ls_type:
        ls_type.append(Type)
N_element = len(ls_type)    # Number of elements

N_type = []    # List of number of atoms of each atom type
for atom_type in ls_type:
    N = 0
    for line in log_lines[start:]:
        Type = line[1]
        if Type == atom_type:
            N += 1
    N_type.append(N)
N_type = ' '.join(str(n) for n in N_type)

F = 'F F F'    # Selective dynamics
T = 'T T T'

fix = input('Selective dynamics? Yes or No : ')
if fix == 'Yes':
    print('The fixed atoms are assumed to be numbered consecutively (e.g. height-by-height). Please quit now if not so.')
    header = '%s\n%f\n%s\n%s\n%s\n%s\n%s\n%s\n%s\n'%(sys, scale, a, b, c, ls_element, N_type, 'Selective dynamics', 'Direct')
    fix_range = input('Enter the IDs of the first and last atom to be fixed, separated by space : ')
    fix_range = fix_range.split()
    fix_range = np.array(fix_range).astype(int)
else:
    header = '%s\n%f\n%s\n%s\n%s\n%s\n%s\n%s\n'%(sys, scale, a, b, c, ls_element, N_type, 'Direct')

print('System information extracted.\n')

## Convert to direct coordinates & print

In [ ]:
file_poscar = pwd + '/POSCAR'
poscar = open(file_poscar,'w')
poscar.write(header)

for atom_type in ls_type:    # List atoms element-by-element
    for line in log_lines[start:]:
        
        ID = line[0]
        Type = line[1]
        
        if Type == atom_type:

            x = line[2]
            y = line[3]
            z = line[4]
            r = np.array([x, y, z])    # Cartesian coordinates
            Rep = np.dot(r, np.linalg.inv(lat))    # Direct coordinates
                        
            if fix == 'Yes':    # Selective dynamics
                if fix_range[0] <= ID <= fix_range[1]:
                    coord = '%f %f %f %s\n'%(Rep[0], Rep[1], Rep[2], F)
                    poscar.write(coord)
                else:
                    coord = '%f %f %f %s\n'%(Rep[0], Rep[1], Rep[2], T)
                    poscar.write(coord)
                            
            else:
                coord = '%f %f %f\n'%(Rep[0], Rep[1], Rep[2])
                poscar.write(coord)

poscar.close()

print('POSCAR generated > ./POSCAR')